<a href="https://colab.research.google.com/github/shahina-darion/spam_mail_detection_model/blob/main/spam_mail_detectio_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
  #  Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [7]:
import pandas as pd

# Read with correct columns
df = pd.read_csv("spam.csv", encoding="latin-1", usecols=[0,1])
df.columns = ["label", "message"]

print(df.head())


                                               label  \
0                 ham\tOk lar... Joking wif u oni...   
1  spam\tFree entry in 2 a wkly comp to win FA Cu...   
2  ham\tU dun say so early hor... U c already the...   
3              ham\tNah I don't think he goes to usf   
4  spam\tFreeMsg Hey there darling it's been 3 we...   

                        message  
0                           NaN  
1                           NaN  
2                           NaN  
3   he lives around here though  
4                 Â£1.50 to rcv  


In [8]:
df = pd.read_csv("spam.csv", sep="\t", names=["label", "message"])


In [9]:
# Drop empty rows
df = df.dropna(subset=["label", "message"]).reset_index(drop=True)

# Map labels
df["label"] = df["label"].map({"ham": 0, "spam": 1})
df = df.dropna(subset=["label"]).reset_index(drop=True)

print("Messages left:", len(df))

# Vectorize
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english", max_features=3000)
X = vectorizer.fit_transform(df["message"]).toarray()
y = df["label"].astype(int).values


Messages left: 5572


In [11]:
from sklearn.model_selection import train_test_split

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)


Shapes: (4457, 3000) (1115, 3000) (4457,) (1115,)


In [12]:
#  Train Logistic Regression Model
model = LogisticRegression(max_iter=200, solver='liblinear')
model.fit(X_train, y_train)

LogisticRegression(max_iter=200, solver='liblinear')

In [13]:
#  Predict
y_pred = model.predict(X_test)

In [14]:
# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9713004484304932

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       966
           1       1.00      0.79      0.88       149

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115



In [15]:
#  Test on New Messages
def predict_message(msg):
    vec = vectorizer.transform([msg]).toarray()
    pred = model.predict(vec)[0]
    return "Spam" if pred == 1 else "Ham"

print("\nSample Predictions:")
print("Message: 'Congratulations! You won a free ticket.' ->", predict_message("Congratulations! You won a free ticket."))
print("Message: 'Hi, are we meeting tomorrow?' ->", predict_message("Hi, are we meeting tomorrow?"))



Sample Predictions:
Message: 'Congratulations! You won a free ticket.' -> Ham
Message: 'Hi, are we meeting tomorrow?' -> Ham
